In [0]:
%sql
with experiment_users as (

select c.userid, b.assignedvalue as bucket, min(b.createdonutc) as entrytime
--b.featurekey as experimentname, c.userid, b.assignedvalue as e_group, min(b.createdonutc) as entrytime
  from
  experimentassignment.featurevariableassignedvalue b
  INNER JOIN experimentassignment.participantidentity c ON b.participantidentityid = c.participantidentityid
  where 
  (b.featurekey='ern2158-mlv6-sunset-ifmodel-v1' OR 
  b.featurekey = 'ern2159-mlv5-experiment-ifmodel-v1' OR
  b.featurekey = 'ml-core-e144-dwell-time-grant-v1')
  AND b.variablekey = 'group-assignment'
  AND b.decisionsource = 'feature-test'
  group by 1,2
),dwell as (select distinct userid
from earnings_engg.offline_grants_ledger_grant_if_expts_log_ml
where ExperimentGroup in ('variant-dwell-time') and MLModelVersion = 'dwell'
and evaluationTimeEst>date'2025-09-24') 
-- truev5 as 
-- (select distinct userid
-- from earnings_engg.offline_grants_ledger_grant_if_expts_log_ml
-- where ExperimentGroup in ('variant-v5') and MLModelVersion = 'v5'
-- and evaluationTimeEst>date'2025-09-24')


select distinct case when eu.bucket is null then 'control' else eu.bucket end as bucket, ta.user_id, eu.entrytime, ta.ticket_created_at_est, ta.ticket_id, ta.contact_driver, case when v5.userid IS null then 'notv5' else 'truev5' end as v5status
from customer_care_dm.fact_user_ticket_all ta
left join experiment_users eu ON eu.userid = ta.user_id AND date(ta.ticket_created_at_est) >= date(eu.entrytime)
inner join dwell v5 ON v5.userid = ta.user_id
where date(ticket_created_at_est) >= date '2025-09-28'
and exclude_flag = 0
and contact_driver LIKE '%earning%'
order by rand ()

In [0]:
import datetime

In [0]:
user_id = 6269644
target_date=None
if target_date is None:
    target_date = datetime.datetime.now().date()

query = """
    select * from main_prod.bankfeed.userprovidedbankconnection where userid = "{}" and isactive = true and connectionstate = 1000
    """.format(user_id)
df = spark.sql(query)
df_pd = df.toPandas()
connection = "inactive"
last_healty_at_list = df_pd['lasthealthyat'].values
created_on_list = df_pd['createdon'].values

for healthy_date, created_on in zip(last_healty_at_list, created_on_list):
    created_on_date = datetime.datetime.strptime(created_on, "%Y-%m-%d %H:%M:%S").date()
    if created_on_date < target_date:
        connection = "active"

print(connection)

In [0]:
df_pd

In [0]:

%sql
select * from main_prod.paycheckprediction.receivedpaycheck where userid  = "6048564"

In [0]:
%sql
select * from main_prod.earnings_analysis.fact_user_earnings_daily where userid  = "6048564" and paydate> target_date and prev_paydate<=target_date

In [0]:
userid = 6048564
target_date = datetime.datetime.now().strftime('%Y-%m-%d')
query = """
select paydate, prev_paydate, prev_payrollactivestatusid,prev_pck_amt, total_pck_amt from main_prod.earnings_analysis.fact_user_earnings_daily where userid  = {} and paydate> '{}' and prev_paydate<='{}'
""".format(userid, target_date, target_date)

query

In [0]:
df = spark.sql(query).distinct()
df_pd = df.toPandas()
df_pd